In [ ]:
#| hide
from hf_clean_benchmarks.core import *

# hf_clean_benchmarks

> This repository contains code for cleaning your training data of benchmark data to help combat data snooping.

This file will become your README and also the index of your documentation.

## Install

```sh
pip install hf_clean_benchmarks
```

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2